In [1]:
!pip install langchain_groq


In [2]:
from langchain_groq import ChatGroq
llm=ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile",
)
result=llm.invoke("What is the capital of France?")
print(result.content)

The capital of France is Paris.


In [3]:
!pip install pypdf

In [4]:
!pip install chromadb

In [5]:
!pip install --upgrade huggingface_hub


In [6]:
!pip install --upgrade sentence-transformers


In [7]:
!pip install chromadb


In [8]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader ,DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import embeddings
from torch import chunk

def initialize_llm():
    llm=ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile",
)
    return llm

def create_vector_db():
    loader=DirectoryLoader(r"C:\Users\manas\Desktop\MentalHealthGen\data", glob="**/*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts=text_splitter.split_documents(documents)
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db=Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("Chroma DB created and persisted.")
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever=vector_db.as_retriever(search_kwargs={"k": 1})
    prompt_template=PromptTemplate(
        template="You are a helpful assistant. Answer the question based on the context provided. {context} Question: {question}",
        input_variables=["context", "question"]
    )
    chain_type_kwargs={"prompt": prompt_template}
    qa_chain=RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )
    return qa_chain


def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.invoke(query)
    print(f"Chatbot: {response['result']}")  # Extracting only the result

    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()


Intializing Chatbot.........


C:\Users\manas\AppData\Local\Temp\ipykernel_20864\4059646782.py:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\manas\AppData\Local\Temp\ipykernel_20864\4059646782.py:26: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


Chroma DB created and persisted.
Chatbot: To achieve happiness, it's essential to address the underlying states of distress that can lead to unhappiness. Considering the context provided, here are some steps to help you cultivate happiness:

1. **Address chronic adversity and stress**: Identify the sources of long-term suffering and stress in your life, and develop strategies to manage or mitigate them. This could involve seeking support from loved ones, therapy, or making lifestyle changes.
2. **Process and cope with loss**: If you're experiencing a reaction to loss, allow yourself to grieve and process your emotions. Seek support from others, and consider professional help if needed.
3. **Build self-esteem**: Focus on your strengths, accomplishments, and positive qualities. Practice self-compassion, and challenge negative self-talk by reframing it in a more positive and realistic light.
4. **Cultivate a optimistic outlook**: While it's essential to acknowledge and validate your emoti

KeyboardInterrupt: Interrupted by user

In [9]:
create_vector_db

<function __main__.create_vector_db()>